# Usable Data Mask (UDM2) Visualization and Masking

The Usable Data Mask (UDM2) is an iteration upon the Unusable Data Mask (UDM) product. It is available for many products and it contains the UDM as one of its bands. Specifications for the UDM2 are given in the [Imagery Spec Sheet](https://assets.planet.com/docs/Planet_Combined_Imagery_Product_Specs_letter_screen.pdf) and the [UDM2 API documentation](https://developers.planet.com/docs/api/udm-2/).

The UDM2 is a multi-band image. The first six bands are boolean bands that encode the absence or presence of different image usability concerns such as haze, clouds, and cloud shadows. The seventh band provides the confidence in the usability assessments as integer values between 0 and 100. The eighth band is the Unusable Data Mask, which is described in the Imagery Spec Sheet referenced above and explored in the [UDM notebook](./udm.ipynb).

In this notebook, we demonstrate how to visualize the UDM2 bands and how to convert the UDM2 into two masks: masking all unclear pixels and masking cloud and shadow pixels.

### Note on UDM2.1

After November 2023, with the launch of UDM2.1, the heavy haze class has been deprecated. A new computer vision model trained at Planet returns more accurate class masks, and includes only a single haze class. For backwards compatibility, `light_haze` will store this haze detection, whereas `heavy_haze` will return zero for all pixels going forward.

## Install Dependencies and Set Up Notebook

In [ ]:
from collections import namedtuple, OrderedDict
import os

import matplotlib
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import rasterio

from planet import Session, DataClient, Auth

%matplotlib inline

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.getenv('PL_API_KEY', 'PASTE_YOUR_KEY_HERE')

client = Auth.from_key(API_KEY)

## Define Scene

The UDM2 asset is available for the `PSScene` item-type. Not all PSScene 4-Band scenes have a UDM2 asset, so here we define the scene and item-type for the UDM2 asset.

In [ ]:
item_type = 'PSScene'
item_id = '20231103_101837_21_2455'

## Visualize Metadata

Visualize the metadata for the scene `PSScene` item-type.

In [ ]:
from planet import data_filter

sfilter = data_filter.and_filter([
    data_filter.permission_filter(),
    data_filter.string_in_filter('id', [item_id])
])

In [ ]:
sfilter

In [ ]:
async def query():
    async with Session() as sess:
        cl = sess.client('data')
        items = [i async for i in cl.search(['PSScene'], sfilter)]
        return items

metadata = await query()

In [ ]:
metadata

We can also search by other fields using other [Search Filters](https://developers.planet.com/docs/apis/data/searches-filtering/), e.g. by date range, geometry, etc.

## Visualize Scene

In [ ]:
# create the data folder if it doesn't exist
data_folder = 'data'
if not os.path.isdir(data_folder): os.mkdir(data_folder)

In [ ]:
# define a helper function for activating and downloading image assets
async def download_asset(item_type, item_id, asset_type, destination_folder, overwrite=True):
    cl = DataClient(Session())

    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type, item_id=item_id, asset_type_id=asset_type)

    # Activate Asset
    await cl.activate_asset(asset=asset_desc)

    # Wait for asset to become active
    print('Awaiting asset activation...', end=' ')
    asset = await cl.wait_asset(asset_desc)

    # Download Asset
    print('Done. Downloading asset.')
    asset_path = await cl.download_asset(asset, directory=destination_folder, overwrite=overwrite)
    
    return asset_path

In [ ]:
asset_type = 'basic_analytic_4b' # for the orthorectified asset, use ortho_analytic_4b
destination_folder = 'data'

img_file = await download_asset(item_type, item_id, asset_type, destination_folder)

In [ ]:
# store the scene filename for processing and make sure the file exists
scene_filename = os.path.join('data', item_id + '_1B_AnalyticMS.tif')
print(scene_filename)
assert os.path.isfile(scene_filename)

In [ ]:
# Utility functions: loading an image
NamedBands = namedtuple('NamedBands', 'b, g, r, nir')

def load_masked_bands(filename):
    """Loads a 4-band BGRNir Planet Image file as a list of masked bands.
    
    The masked bands share the same mask, so editing one band mask will
    edit them all."""
    with rasterio.open(filename) as src:
        b, g, r, nir = src.read()
        mask = src.read_masks(1) == 0 # 0 value means the pixel is masked
    
    bands = NamedBands(b=b, g=g, r=r, nir=nir)
    return NamedBands(*[np.ma.array(b, mask=mask)
                        for b in bands])


print(load_masked_bands(scene_filename).b.shape)

In [ ]:
# Utility functions: displaying an image
def _linear_scale(ndarray, old_min, old_max, new_min, new_max):
    """Linear scale from old_min to new_min, old_max to new_max.
    
    Values below min/max are allowed in input and output.
    Min/Max values are two data points that are used in the linear scaling.
    """
    #https://en.wikipedia.org/wiki/Normalization_(image_processing)
    return (ndarray - old_min)*(new_max - new_min)/(old_max - old_min) + new_min


def _mask_to_alpha(bands):
    band = np.atleast_3d(bands)[...,0]
    alpha = np.zeros_like(band)
    alpha[~band.mask] = 1
    return alpha


def _add_alpha_mask(bands):
    return np.dstack([bands, _mask_to_alpha(bands)])


def bands_to_display(bands, alpha=True):
    """Converts a list of bands to a 3-band rgb, normalized array for display."""
    rgb_bands = np.dstack(bands[:3])

    old_min = np.percentile(rgb_bands, 2)
    old_max = np.percentile(rgb_bands, 98)
    new_min = 0
    new_max = 1
    scaled = _linear_scale(rgb_bands.astype(np.double),
                           old_min, old_max, new_min, new_max)
    bands = np.clip(scaled, new_min, new_max)
    if alpha is True:
        bands = _add_alpha_mask(bands)
    return bands

In [ ]:
fig = plt.figure(figsize=(10, 10))
bands = load_masked_bands(scene_filename)
plt.imshow(bands_to_display([bands.r, bands.g, bands.b]))

plt.title('Scene')
ax = plt.axis('off')

## Download UDM2

To download the UDM2, we use the planet CLI because it handles activating, waiting for activation, and downloading the file.

We will save the UDM2 in the data folder. This folder isn't tracked by git so the downloaded image file will not bloat our git repository.

In [ ]:
udm_file = await download_asset(item_type, item_id, 'ortho_udm2', destination_folder)

In [ ]:
# store the udm2 filename for processing and make sure the file exists
udm2_filename = os.path.join('data', item_id + '_3B_udm2.tif')
print(udm2_filename)
assert os.path.isfile(udm2_filename)

## Load UDM2

In this section, we implement functionality for loading the UDM2 and identifying the bands. We load the UDM2 used in this notebook and determine the percentage of pixels that are clear of any usability concerns.

In [ ]:
# Utility functions for loading a UDM2 image and identify bands as class labels
def load_udm2(udm2_filename):
    '''Load multi-band UDM2 as a 3d array.'''
    with rasterio.open(udm2_filename, 'r') as src:
        udm2 = src.read()
    return udm2


udm2_labels = ['clear', 'snow', 'shadow', 'light haze', 'heavy haze', 'cloud',
               'confidence', 'unusable pixels']

udm2 = load_udm2(udm2_filename)
print(udm2.shape)
print(udm2_labels)

In [ ]:
num_pixels = np.size(udm2[0,:])
num_clear = np.count_nonzero(udm2[0,:])
            
percent_clear = 100 * (num_pixels - num_clear) / num_pixels
print("{0:.01f}% of pixels are considered 'clear'".format(percent_clear))

# Visualize UDM2

### Visualizing a Classified Band

In this section we develop the capability to visualize a classified band, as UDM2 bands are classified bands.

In [ ]:
def plot_classified_band(class_band, class_labels=None,
                         cmap='rainbow',
                         title='Class Labels', figdim=10):

    fig = plt.figure(figsize=(figdim, figdim))
    ax = fig.add_subplot(1, 1, 1)
    imshow_class_band(ax, class_band, class_labels=class_labels, cmap=cmap)
    ax.set_title(title)
    ax.set_axis_off()

    
def imshow_class_band(ax, class_band, class_labels=None, cmap='rainbow'):
    """Show classified band with colormap normalization and color legend.
    Alters ax in place.
    
    possible cmaps ref:
    https://matplotlib.org/examples/color/colormaps_reference.html
    """
    class_norm = _ClassNormalize(class_band)
    im = ax.imshow(class_band, cmap=cmap, norm=class_norm)

    try:
        # add class label legend
        # https://stackoverflow.com/questions/25482876
        # /how-to-add-legend-to-imshow-in-matplotlib
        color_mapping = class_norm.mapping
        colors = [im.cmap(color_mapping[k])
                  for k in class_labels.keys()]
        labels = class_labels.values()

        # https://matplotlib.org/users/legend_guide.html
        # tag: #creating-artists-specifically-for-adding-to-the-legend-aka-proxy-artists
        patches = [mpatches.Patch(color=c, label=l)
                   for c,l in zip(colors, labels)]

        ax.legend(handles=patches, bbox_to_anchor=(1, 1),
                  loc='upper right', borderaxespad=0.)
    except AttributeError:
        # class_labels not specified
        pass


# Inspired by
# https://matplotlib.org/users/colormapnorms.html#custom-normalization-two-linear-ranges
class _ClassNormalize(colors.Normalize):
    """Matplotlib colormap normalizer for a classified band."""
    def __init__(self, arry):
        # get unique unmasked values
        values = [v for v in np.unique(arry)
                  if not isinstance(v, np.ma.core.MaskedConstant)]

        # map unique values to points in the range 0-1
        if len(values) > 1:
            color_ticks = np.array(range(len(values)), dtype=float) / (len(values) - 1)
        else:
            color_ticks = np.array([0])

        self._mapping = dict((v, ct) for v, ct in zip(values, color_ticks))
        
        # Initialize base Normalize instance
        vmin = 0
        vmax = 1
        clip = False
        colors.Normalize.__init__(self, vmin, vmax, clip)
    
    def __call__(self, arry, clip=None):
        '''Create classified representation of arry for display.'''
        # round array back to ints for logical comparison
        arry = np.around(arry)
        new_arry = arry.copy()
        for k, v in self._mapping.items():
            new_arry[arry==k] = v
        return new_arry
    
    @property
    def mapping(self):
        '''property required for colors.Normalize classes
        
        We update the _mapping property in __init__ and __call__ and just
        return that property here.
        '''
        return self._mapping
    

# test out classified band visualization
test_classified_band = np.array(range(9)).reshape((3,3))
plot_classified_band(test_classified_band, title='Test Classified Band',
                     figdim=2)

### Visualize UDM2 Bands

The UDM2 product consists of multiple bands. The first six bands are boolean, present/absent. The seventh band is greyscale, and the eighth is the original bit-encoded UDM.

In [ ]:
def visualize_udm2(udm2, figsize=(15,15)):
    fig, axes = plt.subplots(nrows=4, ncols=2,
                                   sharex=True, sharey=True,
                                   figsize=figsize,
                            constrained_layout=True)
    
    bands = [np.squeeze(b) for b in 
             np.split(udm2, udm2.shape[0], axis=0)]
    band_labels = udm2_labels

    fig.suptitle('UDM2 Bands', fontsize=20)
    bal = zip(bands, axes.flat, udm2_labels)
    for i, (band, ax, label) in enumerate(bal):
        unique = np.unique(band)
    
        # it is unwieldy to show a legend for more than 5 values
        if len(unique) < 5:
            class_labels = OrderedDict((v, v) for v in np.unique(band))
        else:
            class_labels = None
        imshow_class_band(ax, band, class_labels=class_labels, cmap='Greys_r')
        ax.set_title(label)
        ax.set_axis_off()

visualize_udm2(udm2)

The clear band is a combination of all of the other bands. It identifies the snow, clouds, cloud shadows, and light and heavy haze. There appears to be no heavy haze and just a touch of light haze and snow. Really, though, this is a picture of San Francisco, so light haze is possible but snow is unlikely.

### Visualize UDM2 Unusable Data Mask Band

The Unusable Data Mask band of the UDM2 is the original UDM. It is a bit-encoded representation of whether a pixel is usable and, if not, why. Let's check out the usability of the pixels in this image. This section pulls functionality from the [UDM notebook](../udm/udm.ipynb).

In [ ]:
def get_udm_labels(udm):
    '''Get the interpretation of the UDM binary values'''    
    def get_label(v):
        if v == 0:
            label = 'clear'
        elif v == 1:
            label = 'blackfill'
        else:
            labels = []
            if v & int('10', 2):
                labels.append('cloud')
            if v & int('11111100', 2):
                bands = []
                if v & int('100', 2):
                    bands.append('Blue')
                if v & int('1000', 2):
                    bands.append('Green')
                if v & int('10000', 2):
                    bands.append('Red')
                if v & int('100000', 2):
                    bands.append('Red-Edge')
                if v & int('1000000', 2):
                    bands.append('NIR')
                if v & int('10000000', 2):
                    bands.append('Coastal Blue/Green I/Yellow')
                labels.append('missing/suspect {} data'.format(', '.join(bands)))
                
            if not len(labels):
                labels.append('{0:08b}'.format(v))

            label = ' + '.join(labels)
        return label

    return OrderedDict((v, get_label(v)) for v in np.unique(udm))

udm2_udm = udm2[-1, ...]
print('Labels present in this UDM2 UDM band')
get_udm_labels(udm2_udm)

In [ ]:
def visualize_udm2_unusable(udm2, figsize=(15,15)):
    '''Visualize the unusable data mask band of the UDM2 product.'''
    fig, ax = plt.subplots(nrows=1, ncols=1,
                                   sharex=True, sharey=True,
                                   figsize=figsize)
    band = udm2[-1, :]
    label = udm2_labels[-1]

    imshow_class_band(ax, band, class_labels=get_udm_labels(band), cmap='rainbow')
    ax.set_title(label)
    ax.set_axis_off()

    plt.tight_layout()

visualize_udm2_unusable(udm2)

Most of the pixels within the footprint of the image are usable (purple). Most of those that aren't usable are identified as clouds (teal). Just a few pixels have suspect RGB data, likely due to saturation.

## Convert UDM2 to Mask

Converting the UDM2 to a binary mask is pretty straightforward. If one is just interested in only the 'clear' pixels, then they can use the first band of the UDM2 (clear band). The same can be said for avoiding 'cloud' pixels, 'haze' pixels, etc. Masking cloud and cloud shadow pixels is a little more complicated, but still pretty simple. Here, we show how to create a mask that masks all unclear pixels and how to create a mask that masks all cloud and shadow pixels.

### Mask all unclear pixels

In [ ]:
mask_class_labels = {0: 'unmasked', 1: 'masked'}
mask_cmap = 'viridis' # looks better when just two colors are displayed

In [ ]:
def mask_unclear(udm2_array):
    '''Create a mask from the udm2, masking all unclear pixels''' 
    return udm2_array[0,...] == 0

plot_classified_band(mask_unclear(udm2),
                     class_labels=mask_class_labels,
                     cmap=mask_cmap,
                     title='Mask All Unclear Pixels')

### Mask cloud and shadow pixels

In [ ]:
def mask_cloud_shadow(udm2_array):
    '''Create a mask from the udm2, masking all cloud and cloud shadow pixels'''
    shadow_band = udm2_array[2,...]
    cloud_band = udm2_array[5,...]
    masked_pixels = np.logical_or(shadow_band == 1, cloud_band == 1)
    return masked_pixels

plot_classified_band(mask_cloud_shadow(udm2),
                     class_labels=mask_class_labels,
                     cmap=mask_cmap,
                     title='Mask All Cloud and Shadow Pixels')